# Automatic Differentiation

In [2]:
import torch

## A Simple Function

In [3]:
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [4]:
# To calculate the gradient, need a place to store it
x.requires_grad_(True)
x.grad

In [5]:
# calculate our function of x and assign the result to y
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

In [6]:
# access the gradient via x's grad attribute
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [7]:
x.grad == 4 * x

tensor([True, True, True, True])

In [8]:
# PyTorch does not automatically reset the gradient buffer when we record a new gradient
x.grad.zero_() # Reset the gradient
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

## Backward for Non-Scalar Variables

In [9]:
# set the non-scalar tensor's gradient (if scalar, default = 1.0)
x.grad.zero_()
y = x * x
y.backward(gradient=torch.ones(len(y)))  # Faster: y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

## Detaching Computation

In [11]:
# detach the respective computational graph from the final result
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [12]:
# calculate the gradient of y with respect to x
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

## Gradients & Python Control Flow

In [13]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
        
    return c

In [15]:
# realize a specific computational graph and can subsequently run backward
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

In [16]:
# linear function of a with piecewise defined scale
a.grad == d / a

tensor(True)